In [1]:
import pandas as pd
import yaml
import os
from pathlib import Path

# it is helpful to prototype your code running from the same path you
# will be running your data application. In this case, it will be from
# the main directory
if Path(os.getcwd()).name != "data-nerds-tool-survey-results":
    os.chdir("../")

In [2]:
with open("config/env.yml", "r") as stream:
    try:
        env_config = yaml.safe_load(stream)
        for var in env_config["env_var"]:
            os.environ[var] = env_config["env_var"][var]
    except yaml.YAMLError as exc:
        print(exc)
os.environ["survey_results"]

'data/survey/survey_results.csv'

In [3]:
columns = ["timestamp", "role", "years_experience", "programming_language", "programming_package", "data_tool", "cloud_provider"]
survey_df = pd.read_csv(os.environ["survey_results"], names=columns, skiprows=1)
# do not care about the timestamp column
survey_df = survey_df.drop(columns="timestamp")
survey_df.head(1)

,role,years_experience,programming_language,programming_package,data_tool,cloud_provider
0,Data Analytics/Business Intelligence,5+ years,"Python, R, SQL","pandas, matplotlb, seaborn","Tableau, Visual Studio Code, Atom, DBeaver",Amazon Web Services


In [4]:
role_map_df = pd.read_csv(os.environ["role_mapping"])
role_map_df.head(1)

,orig_role,mapped_role
0,"Search Relevance, so it's hard to classify",Other


In [5]:
prog_language_map_df = pd.read_csv(os.environ["programming_language_mapping"])
prog_language_map_df.head(1)

,orig_language,mapped_language
0,solr streaming expressions,solr


In [6]:
prog_package_map_df = pd.read_csv(os.environ["programming_package_mapping"])
prog_package_map_df.head(1)

,orig_package,mapped_package
0,jupyter notebooks,jupyter


In [7]:
data_tool_mapping_df = pd.read_csv(os.environ["data_tool_mapping"])
data_tool_mapping_df.head(1)

,orig_tool,mapped_tool
0,pandas,NaN


In [8]:
cloud_provider_mapping_df = pd.read_csv(os.environ["cloud_provider_mapping"])
cloud_provider_mapping_df.head(1)

,orig_cloud_provider,mapped_cloud_provider
0,custom-built workstations,on-prem


In [9]:
# write a function that takes in a dataframe, a mapping df, and map each of the columns

def map_values_for_column(df, target_column, mapped_df):
    for index, row in mapped_df.iterrows():
        df[target_column] = df[target_column].replace([row[0]], row[1])

    return df

In [10]:
def standardize_survey_list(row):
    # lowers all strings and strips whitespace
    clean_row = [x.lower().strip() for x in row]
    return list(set(clean_row))

In [11]:
def process_survey_results(df, survey_column, mapping_df=None):
    df = df[["role", "years_experience", survey_column]]

    # convert survey column into an array
    df = df.assign(survey_column=df[survey_column].str.split(','))

    # only keep survey columns that have populated values
    df = df[pd.notnull(df["survey_column"])]

    # standardize all of the survey results
    df["survey_column"] = df.apply(lambda row: standardize_survey_list(row["survey_column"]), axis = 1)

    df = df.explode("survey_column")
    
    # reset index and rename column to the survey column
    # name provided
    df = (
        df.reset_index(drop=True)
          .drop(columns=survey_column)
          .rename(columns={"survey_column": survey_column})
    )

    # replace mapped values
    if mapping_df is not None:
        df = map_values_for_column(df, target_column = survey_column, mapped_df=mapping_df)

    # get rid of null and empty columns
    df = df[df[survey_column].notnull()]
    df = df[df[survey_column] != '']

    df = df.groupby(list(df.columns)).size().reset_index(name='count')

    return df

programming_language_survey_df = process_survey_results(survey_df,survey_column="programming_language", mapping_df=prog_language_map_df)
programming_package_survey_df = process_survey_results(survey_df,survey_column="programming_package", mapping_df=prog_package_map_df)
data_tools_survey_df = process_survey_results(survey_df,survey_column="data_tool", mapping_df=data_tool_mapping_df)
cloud_provider_survey_df = process_survey_results(survey_df,survey_column="cloud_provider", mapping_df=cloud_provider_mapping_df)

In [12]:
survey_df = map_values_for_column(survey_df, "role", role_map_df)
survey_df = survey_df[survey_df["role"].notnull()]
survey_df["role"].unique()

array(['Data Analytics/Business Intelligence', 'Data Engineering',
       'Data Science', 'Other', 'Machine Learning Engineering'],
      dtype=object)

In [13]:
tool_list = programming_language_survey_df["programming_language"].unique()
tool_list.sort()
tool_list

array(['bash', 'c++', 'dax', 'javascript', 'matlab', 'perl', 'python',
       'r', 'sas', 'scala', 'solr', 'spss', 'sql', 'stata', 'vba'],
      dtype=object)

In [14]:
tool_list = programming_package_survey_df["programming_package"].unique()
tool_list.sort()
tool_list

array(['apex', 'bokeh', 'boto3', 'd3.js', 'dask', 'dbt', 'dplyr', 'dvc',
       'fastai', 'flask', 'gensim', 'ggplot', 'ggplot2', 'ipython',
       'java', 'jupyter', 'kedro', 'keras', 'luigi', 'matplotlb',
       'matplotlib', 'mlflow', 'nltk', 'numba', 'numpy', 'optuna', 'os',
       'pandas', 'plotly', 'pyodbc', 'pyspark', 'pytorch', 'scikit-learn',
       'scipy', 'seaborn', 'sf', 'spacy', 'spark.sql', 'sqlalchemy',
       'tensorflow', 'tidyverse', 'vis.js'], dtype=object)

In [15]:
tool_list = data_tools_survey_df["data_tool"].unique()
tool_list.sort()
tool_list

array(['alteryx', 'atom', 'aws athena', 'aws quicksight', 'databricks',
       'datagrip', 'datastudio', 'dbeaver', 'dbt', 'excel', 'juicebox',
       'jupyter-notebooks', 'looker', 'powerbi', 'pycharm', 'qlik',
       'rstudio', 'sqlanalytics', 'sublime', 'tableau', 'ultraedit',
       'visual studio code'], dtype=object)

In [16]:
tool_list = cloud_provider_survey_df["cloud_provider"].unique()
tool_list.sort()
tool_list

array(['amazon web services', 'databricks', 'google cloud services',
       'microsoft azure', 'on-prem', 'sas'], dtype=object)